In [9]:
import pandas as pd
import requests
import numpy as np
from fake_useragent import UserAgent
import datetime as dt
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns=None
pd.set_option('max_colwidth', 120)
pd.set_option('display.width', 500)
pd.options.display.float_format ='{:,.2f}'.format

In [2]:
def get_url(key):
    headers_url = {
        'authority': 'rest.adamas.ru',
        'accept': 'application/json, text/plain, */*',
        'accept-language': 'ru,en;q=0.9',
        'authorization': f'Bearer {key}',
        'origin': 'https://adamas.ru',
        'referer': 'https://adamas.ru/',
        'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Yandex";v="22"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-site',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.167 YaBrowser/22.7.3.822 Yowser/2.5 Safari/537.36',
    }

In [3]:
def get_url(key):
    headers_url = {
        'authority': 'rest.adamas.ru',
        'accept': 'application/json, text/plain, */*',
        'accept-language': 'ru,en;q=0.9',
        'authorization': f'Bearer {key}',
        'origin': 'https://adamas.ru',
        'referer': 'https://adamas.ru/',
        'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Yandex";v="22"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-site',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.167 YaBrowser/22.7.3.822 Yowser/2.5 Safari/537.36',
    }
    i = 0
    links = []
    params = {
        'offset': i,
    }
    response = requests.get('https://rest.adamas.ru/v1/products', headers=headers_url)
    js = response.json()
    total_count = js['meta']['total-count']
    while len(js['data']) > 0:
        for d in js['data']:
            try:
                sizes = [i['id'] for i in d['properties'][0]['values']]
            except:
                sizes = np.nan
            try:
                group = d['category_id']
            except:
                group = np.nan
            try:
                article = d['code']
            except:
                article = np.nan
            try:
                name = d['title']
            except:
                name = np.nan
            try:
                link = 'https://adamas.ru' + d['url']
            except:
                link = np.nan
            try:
                prod_id = d['id']
            except:
                prod_id = np.nan
            links.append({
                'name':name,
                'group_id':group,
                'id':prod_id,
                'article':article,
                'sizes':sizes,                
                'link':link
            })
        i += 42
        params = {
        'offset': i,
        }
        response = requests.get('https://rest.adamas.ru/v1/products', params=params, headers=headers_url)
        js = response.json()
        try:
            js['data']
        except:
            key = input('Введи ключ: ')
            headers_url = {
                'authority': 'rest.adamas.ru',
                'accept': 'application/json, text/plain, */*',
                'accept-language': 'ru,en;q=0.9',
                'authorization': f'Bearer {key}',
                'origin': 'https://adamas.ru',
                'referer': 'https://adamas.ru/',
                'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Yandex";v="22"',
                'sec-ch-ua-mobile': '?0',
                'sec-ch-ua-platform': '"Windows"',
                'sec-fetch-dest': 'empty',
                'sec-fetch-mode': 'cors',
                'sec-fetch-site': 'same-site',
                'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.167 YaBrowser/22.7.3.822 Yowser/2.5 Safari/537.36',
            }
    return links, total_count, key

In [4]:
def get_prods(links, key):
    headers_prod = {
        'authority': 'rest.adamas.ru',
        'accept': 'application/json, text/plain, */*',
        'accept-language': 'ru,en;q=0.9',
        'authorization': f'Bearer {key}',
        'origin': 'https://adamas.ru',
        'referer': 'https://adamas.ru/',
        'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Yandex";v="22"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-site',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.167 YaBrowser/22.7.3.822 Yowser/2.5 Safari/537.36',
    }
    date = dt.datetime.now().date().strftime('%d.%m.%Y')
    data = []
    not_connect = []
    for link in tqdm_notebook(links):
        try:
            r_offer = requests.get(f'https://rest.adamas.ru/v1/products/{link["id"]}', headers=headers_prod)
            for size_id in r_offer.json()['offers']:
                if size_id['active'] == True:
                    offer_id = size_id['offer_id']
                    try:
                        size = size_id['size_id']
                    except:
                        size = np.nan
                    try:
                        price_old = size_id['price_old']
                    except:
                        price_old = np.nan
                    try:
                        price = size_id['price']
                    except:
                        price = np.nan   
                    try:
                        weight = size_id['weight']
                    except:
                        weight = np.nan
                    break
            params = []
            for param in r_offer.json()['properties']:
                try:
                    if len(param['values']) > 1:
                        val = [val_id['id'] for val_id in param['values']]
                    else:
                        val = param['values'][0]['id']
                except:
                    val = param['value']
                params.append({
                    param['title']:val
                })
            data.append({
                'date':date,
                'group':link['group_id'],
                'name':link['name'],
                'article':link['article'],
                'weight':weight,
                'size':size,
                'price_old':price_old,
                'price':price,
                'params':params,
                'url':link['link']
            })
        except:
            not_connect.append(link)
            key = input('Введи ключ: ')
            headers_prod = {
                'authority': 'rest.adamas.ru',
                'accept': 'application/json, text/plain, */*',
                'accept-language': 'ru,en;q=0.9',
                'authorization': f'Bearer {key}',
                'origin': 'https://adamas.ru',
                'referer': 'https://adamas.ru/',
                'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Yandex";v="22"',
                'sec-ch-ua-mobile': '?0',
                'sec-ch-ua-platform': '"Windows"',
                'sec-fetch-dest': 'empty',
                'sec-fetch-mode': 'cors',
                'sec-fetch-site': 'same-site',
                'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.167 YaBrowser/22.7.3.822 Yowser/2.5 Safari/537.36',
            }
    df = pd.DataFrame(data)
    return df

# Примечания:
1. Переменную key нужно искать в параметрах подключения к апи в "headers" в параметре "autorization". Там, в принципе, во всех api по адресу rest.adamas одно значение key на определённый период.
2. Переменная key обновляется примерно раз в полчаса. Её генерирует, как я понял, браузер. Пока автоматически перехватывать значение этой переменной я не научился, а написал парсер так, что если значение изменилось - он останавливает выполнение программы и запрашивает ввод значения переменной key, поэтому надо периодически чекать, не запрашивает ли парсер ввести переменную. Благо, парсер довольно быстро собирает данные, много времени это не отнимет.

In [5]:
# from seleniumwire.webdriver import webdriver 

# driver = webdriver.Chrome()
# driver.get('https://adamas.ru/catalog/earrings/21041507-01-A55-41/')
# auth = '' 

# requests = driver.requests
# for r in requests:
#     auth_key = str(r.headers['authorization'])
#     if 'Bea' in auth_key:
#         auth = auth_key.split()[1]
        
# driver.close()

In [6]:
key = input('Введи ключ: ')
links = get_url(key)

In [7]:
links = list(links)[0]

In [8]:
# driver = webdriver.Chrome()
# driver.get('https://adamas.ru/catalog/earrings/21041507-01-A55-41/')
# auth = '' 

# requests = driver.requests
# for r in requests:
#     auth_key = str(r.headers['authorization'])
#     if 'Bea' in auth_key:
#         auth = auth_key.split()[1]
        
# driver.close()
key = input('Введи ключ: ')
df_adamas = get_prods(links, key)

  0%|          | 0/4327 [00:00<?, ?it/s]

In [11]:
df_adamas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4322 entries, 0 to 4321
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date       4322 non-null   object 
 1   group      4322 non-null   int64  
 2   name       4322 non-null   object 
 3   article    4322 non-null   object 
 4   weight     4322 non-null   float64
 5   size       1911 non-null   float64
 6   price_old  4322 non-null   int64  
 7   price      4322 non-null   int64  
 8   params     4322 non-null   object 
 9   url        4322 non-null   object 
dtypes: float64(2), int64(3), object(5)
memory usage: 337.8+ KB


In [12]:
day_now, month_now, year_now = dt.date.today().day, dt.date.today().month, dt.date.today().year
df_adamas.to_csv(rf'C:\Users\Trenkin.Sergey\Desktop\parsers\workspace\adamas_all_{day_now}_{month_now}_{year_now}.csv', index=False)

In [13]:
df_adamas.to_csv(rf'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\МЮР\parsed_data\adamas_all_{day_now}_{month_now}_{year_now}.csv', index=False)